In [ ]:
import random
import json
from cqftsn5g.modules.Models import Flow, Path
def generate_flows(period_range, payload_range, priority_range, jitter_range, bandwidth_range, flow_type, path_range, numbers):
    # random select period, payload, priority, jitter, bandwidth to generate numbers flows
    flows = []
    
    for i in range(numbers):
        period = random.choice(period_range)
        payload = random.choice(payload_range)
        priority = random.choice(priority_range)
        jitter = random.choice([x for x in jitter_range if x < period])
        bandwidth = random.choice(bandwidth_range)
        latency = period
        path = random.choice(path_range)

        flows.append(Flow(period, payload, priority, latency, jitter, bandwidth, flow_type, path))

    return flows
    

In [ ]:
period_range = [2000, 2500, 4000, 5000, 8000, 10000, 16000]
payload_range = [50, 100, 250, 500]
priority_range = [2, 3, 4]
jitter_range = [500, 1000, 2000, 4000, 8000, 16000]

path_range = ["path1", "path2", "path3"]


tt_flows = generate_flows(period_range, payload_range, priority_range, jitter_range, [0], "TT", path_range, 10) 

In [ ]:
period_range = [10000, 16000, 20000, 32000]
payload_range = [800, 1000, 1200, 1500]
priority_range = [5, 6, 7]
jitter_range = [4000, 5000, 10000]
bandwidth_range = [5000, 10000, 12000, 15000, 20000] # KB/sec
path_range = ["path1", "path2", "path3"]


avb_flows = generate_flows(
    period_range, payload_range, priority_range, jitter_range, bandwidth_range, "AVB", path_range, 10
)


In [ ]:
output = json.dump(tt_flows + avb_flows, open("flows.json", "w"))